# Table-Valued Function Examples

This notebook demonstrates 5 different ways to use the fixed `CreateTableFunctionFromCallable` function to register Python functions as table-valued functions (TVFs) in DuckDB.

In [ ]:
import duckdb
import pandas as pd
import random
from datetime import datetime, timedelta

## Example 1: Simple String List TVF

Register a function that returns a list of tuples containing strings.

In [ ]:

# Create connection
def generate_names(count: int = 5):
    """Generate a list of sample names."""
    names = ["Alice1", "Bob", "Charlie", "Diana", "Eve", "Frank", "Grace"]
    return [(names[i % len(names)], i) for i in range(count)]

with duckdb.connect() as conn:

    # Register the TVF with schema definition
    schema = [["name", "VARCHAR"], ["id", "INT"]]

    tvf = conn.create_table_function(
        name="generate_names",
        callable=generate_names,
        parameters=None,  # Will infer from function signature
        schema=schema,
        return_type="records",  # Return list of tuples
    )

    # Use the TVF in a query
    result = conn.execute("SELECT * FROM generate_names(10)").df()
    print(result)

## Example 2: Multi-Column TVF with Typed Parameters

Create a TVF that returns multiple columns with different data types.

In [ ]:
with duckdb.connect() as conn:

    def create_user_data(num_users: int, start_age: int = 25):
        """Create user data with multiple columns."""
        names = ["Alice", "Bob", "Charlie", "Diana", "Eve"]
        departments = ["Engineering", "Marketing", "Sales", "HR"]

        data = []
        for i in range(num_users):
            name = names[i % len(names)]
            age = start_age + random.randint(0, 15)
            department = departments[i % len(departments)]
            salary = 50000 + random.randint(0, 50000)
            data.append((name, age, department, salary))

        return data

    # Define schema with multiple columns and types
    schema = [
        ("name", "VARCHAR"),
        ("age", "INTEGER"),
        ("department", "VARCHAR"),
        ("salary", "INTEGER"),
    ]

    tvf = conn.create_table_function(
        name="create_user_data",
        callable=create_user_data,
        parameters=None,
        schema=schema,
        return_type="strings",
    )

    # Use with parameters and convert to DataFrame for nice display
    result = conn.execute("SELECT * FROM create_user_data(4, 30)").fetchdf()
    print("User data:")
    display(result)
    result.dtypes

## Example 3: TVF that Processes External Data

Simulate loading and processing data from external sources.

In [ ]:
with duckdb.connect() as conn:

    def load_product_data(category_filter: str = "all", limit: int = 10):
        """Simulate loading product data from external source."""
        categories = ["Electronics", "Books", "Clothing", "Sports"]

        data = []
        for i in range(limit):
            category = categories[i % len(categories)]

            # Apply category filter
            if category_filter != "all" and category.lower() != category_filter.lower():
                continue

            data.append(
                (
                    f"Product_{i + 1}",
                    round(random.uniform(10.0, 100.0), 2),
                    random.randint(1, 100),
                    category,
                )
            )

        return data

    schema = [
        ("product_name", "VARCHAR"),
        ("price", "DOUBLE"),
        ("quantity", "INTEGER"),
        ("category", "VARCHAR"),
    ]

    tvf = conn.create_table_function(
        name="load_product_data",
        callable=load_product_data,
        parameters=None,
        schema=schema,
        return_type="strings",
    )

    # Query with aggregation
    result = conn.execute("""
        SELECT 
            category, 
            COUNT(*) as product_count, 
            ROUND(AVG(price), 2) as avg_price,
            SUM(quantity) as total_quantity
        FROM load_product_data('all', 20)
        GROUP BY category
        ORDER BY product_count DESC
    """).fetchdf()

    print("Product summary by category:")
    display(result)

## Example 4: Time Series Data Generator

Generate time series data with window functions.

In [ ]:
with duckdb.connect() as conn:

    def generate_time_series(
        days: int = 7, start_value: float = 100.0, ticker: str = "ACME"
    ):
        """Generate time series stock data."""
        data = []
        current_date = datetime.now().date()
        current_value = start_value

        for i in range(days):
            date = current_date + timedelta(days=i)
            # Random walk
            current_value += random.uniform(-5, 5)
            current_value = max(0, current_value)  # Don't go negative

            data.append(
                (
                    ticker,
                    date.isoformat(),
                    round(current_value, 2),
                    random.randint(100, 1000),  # Volume
                )
            )

        return data

    schema = [
        ("ticker", "VARCHAR"),
        ("date", "DATE"),
        ("price", "DOUBLE"),
        ("volume", "INTEGER"),
    ]

    tvf = conn.create_table_function(
        name="generate_time_series",
        callable=generate_time_series,
        parameters=None,
        schema=schema,
        return_type="strings",
    )

    # Analyze time series with window functions
    result = conn.execute("""
        SELECT 
            ticker,
            date,
            price,
            volume,
            ROUND(price - LAG(price) OVER (ORDER BY date), 2) as price_change,
            ROUND(AVG(price) OVER (ORDER BY date ROWS BETWEEN 2 PRECEDING AND CURRENT ROW), 2) as moving_avg_3
        FROM generate_time_series(10, 150.0, 'DUCKB')
        ORDER BY date
    """).fetchdf()

    print("Time series with moving averages:")
    display(result)

## Example 5: Dynamic Matrix Generation

Create matrix-like data that can be queried with SQL.

In [ ]:
with duckdb.connect() as conn:

    def create_multiplication_table(size: int = 5, multiplier: float = 1.0):
        """Create a multiplication table as tabular data."""
        data = []
        for i in range(1, size + 1):
            for j in range(1, size + 1):
                data.append(
                    (
                        i,  # row
                        j,  # column
                        round(i * j * multiplier, 2),  # value
                    )
                )

        return data

    schema = [("row_num", "INTEGER"), ("col_num", "INTEGER"), ("value", "DOUBLE")]

    tvf = conn.create_table_function(
        name="create_multiplication_table",
        callable=create_multiplication_table,
        parameters=None,
        schema=schema,
        return_type="strings",
    )

    # Query the matrix data with pivot-like operations
    result = conn.execute("""
        SELECT 
            row_num,
            SUM(CASE WHEN col_num = 1 THEN value END) as col_1,
            SUM(CASE WHEN col_num = 2 THEN value END) as col_2,
            SUM(CASE WHEN col_num = 3 THEN value END) as col_3,
            SUM(CASE WHEN col_num = 4 THEN value END) as col_4,
            SUM(value) as row_total
        FROM create_multiplication_table(4, 1.5)
        GROUP BY row_num
        ORDER BY row_num
    """).pl()

    print("Multiplication table (1.5x multiplier) with row totals:")
display(result)

## Key Features Demonstrated

These examples show how the fixed `CreateTableFunctionFromCallable` supports:

1. **Defined Parameters**: Functions can accept typed parameters that are passed from SQL queries
2. **Schema Definition**: Column names and types are explicitly defined and enforced
3. **Return Types**: Support for both Arrow tables and Python list of tuples
4. **SQL Integration**: TVFs work seamlessly with SQL operations like JOINs, aggregations, and window functions
5. **Type Safety**: Proper type conversion between Python and DuckDB types

The implementation is now clean, maintainable, and follows the same patterns as ScalarUDF while being much simpler than the previous broken implementation.

In [ ]:
# Test the updated TVF implementation with all notebook examples
import duckdb
import pandas as pd
import random
from datetime import datetime, timedelta

print("Testing Example 1: Simple String List TVF")

def generate_names(count: int = 5):
    """Generate a list of sample names."""
    names = ["Alice", "Bob", "Charlie", "Diana", "Eve", "Frank", "Grace"]
    return [(names[i % len(names)], i) for i in range(count)]

# Test with records return type (should work)
conn = duckdb.connect()
schema = [("name", "VARCHAR"), ("id", "INT")]

try:
    tvf = conn.create_table_function(
        name="generate_names",
        callable=generate_names,
        parameters=None,
        schema=schema,
        return_type="records",  # Use records instead of strings
    )
    
    result = conn.execute("SELECT * FROM generate_names(3)").df()
    print("✓ Example 1 with 'records' return type:")
    print(result)
    print()
except Exception as e:
    print(f"✗ Example 1 failed: {e}")
    print()

# Test backward compatibility with strings
try:
    conn.unregister_table_function("generate_names")
    tvf = conn.create_table_function(
        name="generate_names_strings",
        callable=generate_names,
        parameters=None,
        schema=schema,
        return_type="strings",  # Should still work (backward compatibility)
    )
    
    result = conn.execute("SELECT * FROM generate_names_strings(3)").df()
    print("✓ Example 1 with 'strings' return type (backward compatibility):")
    print(result)
    print()
except Exception as e:
    print(f"✗ Example 1 strings backward compatibility failed: {e}")
    print()

conn.close()

In [ ]:
# Test if the connection is being stored properly in the registry
import duckdb
conn = duckdb.connect()

def simple_test():
    return [("test", 1)]

schema = [("name", "VARCHAR"), ("id", "INT")]

try:
    # Try registering and immediately querying
    tvf = conn.create_table_function(
        name="simple_test",
        callable=simple_test,
        parameters=None,
        schema=schema,
        return_type="records"
    )
    
    print("TVF registered successfully")
    
    # Try to use it immediately in the same context
    result = conn.execute("SELECT * FROM simple_test()").fetchall()
    print(f"Result: {result}")
    
except Exception as e:
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()

conn.close()